In [1]:
import numpy as np

n = 10000
x, y = np.random.random((2, n))

In [2]:
def np_cos_norm(a, b):
    val = np.sum(1. - np.cos(a-b))
    return np.sqrt(val / 2. / a.shape[0])

In [3]:
%timeit np_cos_norm(x, y)

135 µs ± 4.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
%load_ext Cython

In [5]:
%%cython
import numpy as np
def cy_np_cos_norm(a, b):
    val = np.sum(1. - np.cos(a-b))
    return np.sqrt(val / 2. / a.shape[0])

In [6]:
%timeit cy_np_cos_norm(x, y)

130 µs ± 2.03 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%%cython
import numpy as np
cimport numpy as np
def cy_np_typed_cos_norm(np.ndarray[double, ndim=1] a, np.ndarray[double, ndim=1] b):
    val = np.sum(1. - np.cos(a-b))
    return np.sqrt(val / 2. / a.shape[0])

In [8]:
%timeit cy_np_typed_cos_norm(x, y)

136 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%%cython
#cython: np_pythran=True
#cython: cxx=True
import numpy as np
cimport numpy as np

def cy_np_typed_pythran_cos_norm(np.ndarray[double, ndim=1] a, np.ndarray[double, ndim=1] b):
    cdef int n = len(a)
    val = np.sum(1. - np.cos(a-b))
    return np.sqrt(val / 2. / n)

In [10]:
%timeit cy_np_typed_pythran_cos_norm(x, y)

125 µs ± 534 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
%%cython

import numpy as np
cimport numpy as np
from libc.math cimport cos, sqrt

cimport cython
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def cy_np_typed_python_cos_norm(np.ndarray[double, ndim=1] a, np.ndarray[double, ndim=1] b):
    cdef int n = len(a), i
    cdef double acc = 0,res
    for i in range(n):
        acc += 1 - cos(a[i]-b[i])
    res = sqrt(acc / 2. / n)
    return res

In file included from /home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1821,
                 from /home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/sguelton/.cache/ipython/cython/_cython_magic_de5fadab911cf21905da8dd16683ccf9.c:663:
/home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^~~~~~~


In [40]:
%timeit cy_np_typed_python_cos_norm(x, y)

127 µs ± 957 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
%%cython -c=-DUSE_XSIMD -c=-march=native
#cython: np_pythran=True
#cython: cxx=True
import numpy as np
cimport numpy as np

def cy_np_typed_pythran_xsimd_cos_norm(np.ndarray[double, ndim=1] a, np.ndarray[double, ndim=1] b):
    cdef int n = len(a)
    val = np.sum(1. - np.cos(a-b))
    return np.sqrt(val / 2. / n)

In [18]:
%timeit cy_np_typed_pythran_xsimd_cos_norm(x, y)

28.1 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
def np_laplacien(image):
    out_image = np.abs(4*image[1:-1,1:-1] -
                       image[0:-2,1:-1] - image[2:,1:-1] -
                       image[1:-1,0:-2] - image[1:-1,2:])
    valmax = np.max(out_image)
    valmax = max(1.,valmax)+1.E-9
    out_image /= valmax
    return out_image

In [32]:
N = 500 ; image = np.random.randn(N,N)

In [33]:
%timeit np_laplacien(image)

1.16 ms ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%%cython
import numpy as np
def cy_np_laplacien(image):
    out_image = np.abs(4*image[1:-1,1:-1] -
                       image[0:-2,1:-1] - image[2:,1:-1] -
                       image[1:-1,0:-2] - image[1:-1,2:])
    valmax = np.max(out_image)
    valmax = max(1.,valmax)+1.E-9
    out_image /= valmax
    return out_image

In [35]:
%timeit cy_np_laplacien(image)

1.37 ms ± 51.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%%cython
#cython: np_pythran=True
#cython: cxx=True
import numpy as np
cimport numpy as np

def cy_np_pythran_laplacien(np.ndarray[double, ndim=2] image):

    out_image = np.abs(4*image[1:-1,1:-1] -
                       image[0:-2,1:-1] - image[2:,1:-1] -
                       image[1:-1,0:-2] - image[1:-1,2:])
    valmax = np.max(out_image)
    valmax = max(1.,valmax)+1.E-9
    out_image /= valmax
    return out_image

In [38]:
%timeit cy_np_pythran_laplacien(image)

572 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
%%cython
import numpy as np
cimport numpy as np
from libc.math cimport fabs

cimport cython
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def cy_py_laplacien(np.ndarray[double, ndim=2] image):
    cdef int i, j
    cdef int n = image.shape[0], m = image.shape[1]
    cdef np.ndarray[double, ndim=2] out_image = np.empty((n-2,m-2))
    cdef double valmax
    for i in range(n-2):
        for j in range(m-2):
            out_image[i,j] =  fabs(4*image[1+i,1+j] -
                       image[i,1+j] - image[2+i,1+j] -
                       image[1+i,j] - image[1+i,2+j])
    valmax = out_image[0,0]
    for i in range(n-2):
        for j in range(m-2):
            if out_image[i,j] > valmax:
                valmax = out_image[i,j]
    valmax = max(1.,valmax)+1.E-9
    for i in range(n-2):
        for j in range(m-2):
            out_image[i,j] /= valmax
    return out_image

In file included from /home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1821,
                 from /home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/sguelton/.cache/ipython/cython/_cython_magic_13f923ceefb5a7aa28145e7decf40983.c:667:
/home/sguelton/.venvs/pythran-stories/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^~~~~~~


In [46]:
%timeit cy_py_laplacien(image)

781 µs ± 3.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
